In [1]:
import pandas as pd
from scipy.signal import find_peaks
import plotly.graph_objects as go
import numpy as np
import itertools
# from peakdetect import peakdetect
import os
import matplotlib.pyplot as plt

In [2]:
# storage_dict = {} # initalise dictonary to save keys = volts_strings and the corresponding_list

# Import data

In [3]:
folders = [0.24,   11.00 , 13.00 , 15.00  ,17.25 , 19.25 , 3.25	,5.50 , 7.75,
0.25 ,  11.25  ,13.25,  15.25,  1.75  , 19.50 , 3.50,	5.75 , 8.00,
0.50 ,  11.50 , 13.50 , 15.50  ,17.50  ,19.75  ,3.75	,6.00 ,8.25,
0.75  , 11.75  ,13.75  ,15.75 , 17.75,  19.99 , 4.00,	6.25,  8.50,
1.00  , 12.00,  14.00 , 16.00 , 18.00 , 2.00  , 4.25,	6.50 , 8.75,
10.00,  12.25 , 14.25,  16.25,  18.25,  2.25,   4.50,	6.75 , 9.00,
10.25,  1.25  , 14.50,  16.50 , 18.50 , 2.50,   4.75,	7.00 , 9.25,
10.50 , 12.50 , 14.75,  16.75,  18.75 , 2.75,   5.00,	7.25  ,9.50,
10.75 , 12.75 , 1.50  , 17.00  ,19.00 , 3.00,   5.25,	7.50,  9.75]

In [4]:
folders = np.sort(folders)
folders

array([ 0.24,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,
        2.25,  2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.25,
        4.5 ,  4.75,  5.  ,  5.25,  5.5 ,  5.75,  6.  ,  6.25,  6.5 ,
        6.75,  7.  ,  7.25,  7.5 ,  7.75,  8.  ,  8.25,  8.5 ,  8.75,
        9.  ,  9.25,  9.5 ,  9.75, 10.  , 10.25, 10.5 , 10.75, 11.  ,
       11.25, 11.5 , 11.75, 12.  , 12.25, 12.5 , 12.75, 13.  , 13.25,
       13.5 , 13.75, 14.  , 14.25, 14.5 , 14.75, 15.  , 15.25, 15.5 ,
       15.75, 16.  , 16.25, 16.5 , 16.75, 17.  , 17.25, 17.5 , 17.75,
       18.  , 18.25, 18.5 , 18.75, 19.  , 19.25, 19.5 , 19.75, 19.99])

In [5]:
path = "/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/"
# file_path_ls = ['/content/drive/MyDrive/PHYS 243W/RLC Chaos Lab/data at 279.800kHz/{0:.2f}/{1:.2f}_05.csv'.format(x,x)  for x in folders]
file_path_ls = [path+'{0:.2f}/{1:.2f}_05.csv'.format(x,x)  for x in folders]
file_path_ls[28] = path+'7.00/7.00_005.csv'
file_path_ls[30] = path + '7.50/7.50_005.csv'
file_path_ls[0]

'/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/0.24/0.24_05.csv'

In [6]:
def threshold_group_maker(l, threshold = 5):
    groups = {}
    its = 0
    for i in l:
        numbers_except = list(l)
        del numbers_except[(l.index(i))]
        for x in range(len(numbers_except)):
            if difference(i, numbers_except[x]) < threshold:
                try:
                    groups[its].append(numbers_except[x])
                except:
                    groups[its] = [numbers_except[x]]
                del l[l.index(numbers_except[x])]
        try:
            groups[its].append(i)
        except:
            groups[its] = [i]
        its += 1
    return(groups)

def difference(a, b):
    return abs(a-b)

In [7]:
list( zip([.5], [.7, 9]) )

[(0.5, 0.7)]

In [8]:
def run_one(file_path, volt_value, difference):
  rlc_data = pd.read_csv(filepath_or_buffer =file_path) # first file

  if rlc_data.columns[0] =='Frequency': # had error from 16.50 onwards
    rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})
  
  time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
  time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float) # output
  time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float) # driving

  #indices = find_peaks(time_series_B,  prominence=1 )[0]
  # indices = find_peaks(time_series_B,  prominence=0, distance=110)[0]
  indices = find_peaks(time_series_B,  prominence=0, distance=20)[0]
  positive_indices = indices[time_series_B[indices] > 0]
  indices = positive_indices

  peaks_channel_B =  [time_series_B[j] for j in indices] # these are the peaks from find_peaks functiom
  peaks_channel_A =  [time_series_A[j] for j in indices] # output voltage corresponding to peaks of driving voltage

  tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
  tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])
  # print(sorted( list( peaks_channel_A) ), len(list( peaks_channel_A)))

  dict_groups = threshold_group_maker(sorted( list( peaks_channel_A) ), threshold=difference )
  # print(dict_groups)
  key_vals = list ( dict_groups.keys() )

  # peaks_channel_A_unique = [ i for i in peaks_channel_A ]
  # for i in range( len(peaks_channel_A) ) :
  #     for j in key_vals:
  #         if peaks_channel_A_unique[i] in dict_groups[j]:
  #             peaks_channel_A_unique[i] = np.mean(dict_groups[j])
  #         else:
  #             continue

  peaks_channel_A_unique = [ np.mean(dict_groups[j]) for j in key_vals ]
  # print(peaks_channel_A_unique)
  
  # tuples_A_B_rounded = list( zip(peaks_channel_A_unique, peaks_channel_B) )
  # print("tuple", tuples_A_B_rounded)
  # tuples_A_B_rounded = sorted(tuples_A_B_rounded, key=lambda x: x[0])
  
  storage_dict = {}
  volts_string = str(volt_value)
  # corresponding_list = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded))]
  
  storage_dict[volts_string] = peaks_channel_A_unique
  return storage_dict

In [9]:
check = run_one(file_path_ls[0],folders[0], difference=0.5)
check

{'0.24': [0.43964089285714286]}

In [10]:
# def run_all(ls_path, ls_volts, difference=1):
#   store_all = {}
#   for i in range(len(ls_path)):
#     path = ls_path[i]
#     volt_value = ls_volts[i]
#     store_one = run_one(file_path = path, volt_value = volt_value, difference = difference)
#     store_all.update(store_one)
  
#   return store_all

def run_all(ls_path, ls_volts, difference_list):
  store_all = {}
  for i in range(len(ls_path)):
    path = ls_path[i]
    volt_value = ls_volts[i]
    
    if volt_value< 1.5:
      difference = difference_list[0]
    elif 1.5<= volt_value < 5:
      difference = difference_list[1]
    elif 5<= volt_value < 10:
      difference = difference_list[2]
    else:
      difference = difference_list[3]
    
    store_one = run_one(file_path = path, volt_value = volt_value, difference = difference)
    store_all.update(store_one)
  
  return store_all

# Plot for different thresholds (when we take two peaks as separate)

In [15]:
# diff_l = [ [0.5, 1, 3, 5], [ 1, 3, 5, 10], [1.5, 3,5,5] ]
diff_l = [ [ 1,2,2,2], [1.5, 3,5,5]] # you can playaround with this

for dist_list in diff_l: 
    results = run_all(file_path_ls,folders, difference_list=dist_list)
    # results
    keys, vals , vals_len= [], [], []
    for k in results.keys():
        keys.append(float(k))
        vals.append(results.get(k))
        vals_len.append(len(results.get(k)))
    
    plt_lsx = []
    for j in range(len(vals)):
        ls = [vals[j][i] for i in range(len(vals[j])) ]
        for k in range(len(ls)):
            plt_lsx.append([float(keys[j]), ls[k]])

    x_axis = [i[0] for i in plt_lsx]  
    y_axis = np.asarray([i[1] for i in plt_lsx] ) 

    trace1 = go.Scatter(
        x = x_axis,
        y = y_axis,
        mode='markers ',   # scatter mode (more in UG section 1)
        opacity = .6,
        marker = dict(size = 8, color = 'blue', line=dict(width=0.5))
        )
    data = [trace1]

    layout = go.Layout(
        autosize=False,
        width=750,
        height=500,

        xaxis= go.layout.XAxis(linecolor = 'black',
                            linewidth = 1,
                            mirror = True),

        yaxis= go.layout.YAxis(linecolor = 'black',
                            linewidth = 1,
                            mirror = True),

        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad = 4
        )
    )
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(title= 'threshold =' + str(dist_list) )
    fig.update_layout(yaxis_title="Output Voltage (mV)", xaxis_title="Driving Voltage (V)")
    fig.show()

In [302]:
# threshold = [0.5, 1, 3, 5]
# for distance in threshold: 
#     results = run_all(file_path_ls,folders, difference=distance)
#     # results
#     keys, vals , vals_len= [], [], []
#     for k in results.keys():
#         keys.append(float(k))
#         vals.append(results.get(k))
#         vals_len.append(len(results.get(k)))
    
#     plt_lsx = []
#     for j in range(len(vals)):
#         ls = [vals[j][i] for i in range(len(vals[j])) ]
#         for k in range(len(ls)):
#             plt_lsx.append([float(keys[j]), ls[k]])

#     x_axis = [i[0] for i in plt_lsx]  
#     y_axis = np.asarray([i[1] for i in plt_lsx] ) 

#     trace1 = go.Scatter(
#         x = x_axis,
#         y = y_axis,
#         mode='markers ',   # scatter mode (more in UG section 1)
#         opacity = .6,
#         marker = dict(size = 8, color = 'blue', line=dict(width=0.5))
#         )
#     data = [trace1]

#     layout = go.Layout(
#         autosize=False,
#         width=750,
#         height=500,

#         xaxis= go.layout.XAxis(linecolor = 'black',
#                             linewidth = 1,
#                             mirror = True),

#         yaxis= go.layout.YAxis(linecolor = 'black',
#                             linewidth = 1,
#                             mirror = True),

#         margin=go.layout.Margin(
#             l=50,
#             r=50,
#             b=100,
#             t=100,
#             pad = 4
#         )
#     )
#     fig = go.Figure(data=data, layout=layout)
#     fig.update_layout(title= 'threshold =' + str(distance) )
#     fig.update_layout(yaxis_title="Output Voltage (mV)", xaxis_title="Driving Voltage (V)")
#     fig.show()

# Check once

In [294]:
results = run_all(file_path_ls,folders, difference=1)
# results

In [295]:
keys, vals , vals_len= [], [], []
for k in results.keys():
  keys.append(float(k))
  vals.append(results.get(k))
  vals_len.append(len(results.get(k)))

In [296]:
plt_lsx = []

for j in range(len(vals)):
  ls = [vals[j][i] for i in range(len(vals[j])) ]
  for k in range(len(ls)):
    plt_lsx.append([float(keys[j]), ls[k]])

x_axis = [i[0] for i in plt_lsx]  
y_axis = np.asarray([i[1] for i in plt_lsx] ) 

In [301]:
trace1 = go.Scatter(
    x =x_axis,
    y = y_axis,
    mode='markers ',   # scatter mode (more in UG section 1)
    # text = label3,
    opacity = 1,
    # textposition='top center',
    marker = dict(size = 8, color = 'blue', line=dict(width=0.5))
    )
data = [trace1]

layout = go.Layout(
        autosize=False,
        width=750,
        height=500,

        xaxis= go.layout.XAxis(linecolor = 'black',
                            linewidth = 1,
                            mirror = True),

        yaxis= go.layout.YAxis(linecolor = 'black',
                            linewidth = 1,
                            mirror = True),

        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad = 4
        )
    )

fig = go.Figure(data=data, layout=layout)
fig.update_layout(title= 'threshold =' + str(distance) )
fig.update_layout(yaxis_title="Output Voltage (mV)", xaxis_title="Driving Voltage (V)")
fig.show()

In [26]:
 # Check
vals[60]

[(-33.432645, -7.130039),
 (-33.432645, -7.129734),
 (-33.432645, -7.092502),
 (-33.432645, -7.087314),
 (-33.432645, -7.086398),
 (-33.432645, -7.08121),
 (-33.432645, -7.078463),
 (-33.432645, -7.066256),
 (-20.62406777777778, -7.1395),
 (-20.62406777777778, -7.135533),
 (-20.62406777777778, -7.127293),
 (-20.62406777777778, -7.123936),
 (-20.62406777777778, -7.111423),
 (-20.62406777777778, -7.105625),
 (-20.62406777777778, -7.066561),
 (-20.62406777777778, -7.056795),
 (-20.62406777777778, -7.043367),
 (-6.938382753333333, -7.132175),
 (-6.938382753333333, -7.128513),
 (-6.938382753333333, -7.123631),
 (-6.938382753333333, -7.119663),
 (-6.938382753333333, -7.061678),
 (-6.938382753333333, -7.050386),
 (19.602217058823527, 7.132786),
 (19.602217058823527, 7.141942),
 (19.602217058823527, 7.14835),
 (19.602217058823527, 7.194738),
 (19.602217058823527, 7.19718),
 (19.602217058823527, 7.201758),
 (19.602217058823527, 7.207556),
 (19.602217058823527, 7.218848),
 (19.602217058823527, 7

In [94]:
rlc_data_1 = pd.read_csv(filepath_or_buffer = path+'5.50/5.50_05.csv') # first file
rlc_data_2 = pd.read_csv(filepath_or_buffer = path+'5.50/5.50_05.csv') # second file
rlc_data_2 = rlc_data_2.drop(axis=0, index=0)

In [95]:
rlc_data = rlc_data_1.append(rlc_data_2, ignore_index=True)
# rlc_data = rlc_data_1

In [96]:
if rlc_data.columns[0] =='Frequency':
  rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})

In [97]:
rlc_data

,Time,Channel A,Channel B
0,(us),(mV),(V)
1,-25.00669250,0.13489180,1.29871500
2,-24.98169250,0.35279390,1.40873400
3,-24.95669251,0.54322940,1.49433900
4,-24.93169251,0.74587230,1.57506000
...,...,...,...
4028,25.26830451,2.06732400,2.10562500
4029,25.29330451,2.29194000,2.17841100
4030,25.31830451,2.48664800,2.25806500
4031,25.34330451,2.62764300,2.30872500


In [98]:
time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)

## Draw from sanity check 

In [99]:
fig = go.Figure(data=go.Scatter(
    x = time_values,
    y = time_series_B,
    mode = 'markers'
))

fig.show()

# Find and draw peaks

In [132]:
indices = find_peaks(time_series_B,  prominence=0, distance=25, threshold=0.05 * 10**(-3))[0]
indices

array([  25,   96,  166,  235,  310,  384,  453,  528,  597,  668,  738,
        807,  878,  956, 1025, 1095, 1166, 1240, 1310, 1379, 1453, 1528,
       1597, 1667, 1738, 1812, 1882, 1951, 2015, 2041, 2112, 2182, 2251,
       2326, 2400, 2469, 2544, 2613, 2684, 2754, 2823, 2894, 2972, 3041,
       3111, 3182, 3256, 3326, 3395, 3469, 3544, 3613, 3683, 3754, 3828,
       3898, 3967])

In [133]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel A'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.show()

## Drawing peaks of A and corresponding values of channel B

In [85]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_A,
    mode='lines+markers',
    name='Channel A'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_A[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel B'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='circle'
    ),
    name='B values corr to Peak A'
))

fig.show()

# Getting the tuple of the values of peaks

In [147]:
peaks_channel_A =  [time_series_A[j] for j in indices]
peaks_channel_B =  [time_series_B[j] for j in indices]

In [148]:
tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])
tuples_A_B

[(-13.18949, -2.640309),
 (-13.18949, -2.640309),
 (-13.15897, -2.641987),
 (-13.15897, -2.641987),
 (-13.01004, -2.624744),
 (-13.01004, -2.624744),
 (-12.97281, -2.643513),
 (-12.97281, -2.643513),
 (-12.79458, -2.622608),
 (-12.79458, -2.622608),
 (-12.62368, -2.641835),
 (-12.62368, -2.641835),
 (-12.62001, -2.618488),
 (-12.62001, -2.618488),
 (-12.48512, -2.606891),
 (-12.48512, -2.606891),
 (-12.40699, -2.641377),
 (-12.40699, -2.641377),
 (-12.35511, -2.641682),
 (-12.35511, -2.641682),
 (-11.87719, -2.622608),
 (-11.87719, -2.622608),
 (-11.86376, -2.619709),
 (-11.86376, -2.619709),
 (-11.84118, -2.617267),
 (-11.84118, -2.617267),
 (-11.76183, -2.624439),
 (-11.76183, -2.624439),
 (2.778405, 2.342143),
 (4.104739, 2.595599),
 (4.104739, 2.595599),
 (4.151738, 2.621693),
 (4.151738, 2.621693),
 (4.596697, 2.622303),
 (4.596697, 2.622303),
 (4.649799, 2.627796),
 (4.649799, 2.627796),
 (4.741965, 2.626881),
 (4.741965, 2.626881),
 (4.884181, 2.628407),
 (4.884181, 2.628407),
 

# Get unique ones 
Notice that 62.5 and 63 are basically the same peak but we cannot just take the unique set because these two would come as different values. So we first write a function where the values are clubbed together if they are within a certain threshold

In [153]:
def define_groups(l, threshold = 15):
    groups = {}
    its = 0
    for i in l:
        numbers_except = list(l)
        del numbers_except[(l.index(i))]
        for x in range(len(numbers_except)):
            if difference(i, numbers_except[x]) < threshold:
                try:
                    groups[its].append(numbers_except[x])
                except:
                    groups[its] = [numbers_except[x]]
                del l[l.index(numbers_except[x])]
        try:
            groups[its].append(i)
        except:
            groups[its] = [i]
        its += 1
    return(groups)

def difference(a, b):
    return abs(a-b)

In [154]:
dict_groups = define_groups(sorted( list( peaks_channel_A) ) )
dict_groups

{0: [-13.18949,
  -13.15897,
  -13.15897,
  -13.01004,
  -13.01004,
  -12.97281,
  -12.97281,
  -12.79458,
  -12.79458,
  -12.62368,
  -12.62368,
  -12.62001,
  -12.62001,
  -12.48512,
  -12.48512,
  -12.40699,
  -12.40699,
  -12.35511,
  -12.35511,
  -11.87719,
  -11.87719,
  -11.86376,
  -11.86376,
  -11.84118,
  -11.84118,
  -11.76183,
  -11.76183,
  -13.18949],
 1: [4.104739,
  4.104739,
  4.151738,
  4.151738,
  4.596697,
  4.596697,
  4.649799,
  4.649799,
  4.741965,
  4.741965,
  4.884181,
  4.884181,
  4.982451,
  4.982451,
  5.006256,
  5.006256,
  5.365764,
  5.365764,
  5.500045,
  5.500045,
  5.68926,
  5.68926,
  5.971862,
  5.971862,
  6.010925,
  6.010925,
  6.252632,
  6.252632,
  2.778405]}

### if they are in the same group replace by the mean

In [59]:
peaks_channel_A[1]

64.391

In [60]:
keys = list ( dict_groups.keys() )

In [61]:
peaks_channel_A_unique = [ i for i in peaks_channel_A ]
for i in range( len(peaks_channel_A) ) :
    for j in keys:
        # print(j)
        # print( peaks_channel_A[i] )
        # print(j)
        if peaks_channel_A_unique[i] in dict_groups[j]:
            peaks_channel_A_unique[i] = np.mean(dict_groups[j])
        else:
            continue

In [62]:
print( "original: ", peaks_channel_A)
print( "unique: ", peaks_channel_A_unique)

original:  [15.40574, 64.391, 28.44935, 23.1727, 46.04633, 16.27247, 66.00848, 29.15128, 17.76177, 46.87948, 16.35487, 66.93014, 27.94275, 21.98858]
unique:  [16.4487125, 65.77654000000001, 28.51446, 22.58064, 46.462905, 16.4487125, 65.77654000000001, 28.51446, 16.4487125, 46.462905, 16.4487125, 65.77654000000001, 28.51446, 22.58064]


In [63]:
tuples_A_B_rounded = list( zip(peaks_channel_A_unique, peaks_channel_B) )
tuples_A_B_rounded = sorted(tuples_A_B_rounded, key=lambda x: x[0])
tuples_A_B_rounded

[(16.4487125, 7.97235),
 (16.4487125, 7.98944),
 (16.4487125, 7.977539),
 (16.4487125, 7.989135),
 (22.58064, 7.978454),
 (22.58064, 7.980896),
 (28.51446, 7.982421),
 (28.51446, 7.975707),
 (28.51446, 7.973266),
 (46.462905, 7.96289),
 (46.462905, 7.975707),
 (65.77654000000001, 7.965026),
 (65.77654000000001, 7.966552),
 (65.77654000000001, 7.957396)]

### finally group the tuples with the same first coordinate together

In [64]:
f = lambda x:x[0]
x = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded, key=f), key=f)]
x

[(16.4487125, 7.97235),
 (22.58064, 7.978454),
 (28.51446, 7.982421),
 (46.462905, 7.96289),
 (65.77654000000001, 7.965026)]

In [65]:
volts_string = '7 Volts'
corresponding_list = [next(g) for _, g in itertools.groupby(sorted(tuples_A_B_rounded, key=f), key=f)]

In [66]:
print("Number of points for the bifurication diagram are : {}".format(len(x)))

Number of points for the bifurication diagram are : 5


## Getting dictionary

In [67]:
storage_dict[volts_string] = corresponding_list

In [68]:
storage_dict

{'7 Volts': [(16.4487125, 7.97235),
  (22.58064, 7.978454),
  (28.51446, 7.982421),
  (46.462905, 7.96289),
  (65.77654000000001, 7.965026)]}